In [48]:
import os
import json
import requests
import pandas as pd
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv
from zipfile import ZipFile
from google.cloud import storage
from google.oauth2 import service_account

load_dotenv()

ROOT = Path.cwd().parent
DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

PROJECT_ID = os.getenv("PROJECT_ID")
SA_PATH = ROOT / os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
BUCKET_NAME = os.getenv("BUCKET_NAME")


In [33]:
# Authentification Google Cloud
creds = service_account.Credentials.from_service_account_file(SA_PATH)
storage_client = storage.Client(project=PROJECT_ID, credentials=creds)

In [34]:
# Utils functions
def download_parquet_from_idfm(dataset_name):
    """Télécharge un fichier Parquet depuis l'API IDFM"""
    URL = f"https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/{dataset_name}/exports/parquet?parquet_compression=snappy"
    OUTPUT_FILE = f"{dataset_name}.parquet"

    dataset_dir = DATA_DIR / dataset_name
    dataset_dir.mkdir(exist_ok=True)
    parquet_path = dataset_dir / OUTPUT_FILE

    print(f"[...] - Téléchargement du fichier Parquet {dataset_name}...")

    response = requests.get(URL, timeout=300)
    response.raise_for_status()

    parquet_path.write_bytes(response.content)

    file_size_mb = parquet_path.stat().st_size / (1024 * 1024)
    print(f"[OK] - Téléchargé: {parquet_path}")
    print(f"[OK] - Taille: {file_size_mb:.2f} MB")
    
    return parquet_path

def upload_to_gcs(file_path, gcs_folder="bronze", gcs_subfolder=""):
    """Upload un fichier local vers Google Cloud Storage"""
    file_path = Path(file_path)
    
    if not file_path.exists():
        raise FileNotFoundError(f"Le fichier {file_path} n'existe pas")
    
    bucket = storage_client.bucket(BUCKET_NAME)
    gcs_path = f"{gcs_folder}/{gcs_subfolder}/{file_path.name}" if gcs_subfolder else f"{gcs_folder}/{file_path.name}"
    blob = bucket.blob(gcs_path)
    
    print(f"[...] - Upload de {file_path.name} vers GCS...")
    blob.upload_from_filename(str(file_path))
    
    file_size_mb = file_path.stat().st_size / (1024 * 1024)
    print(f"[OK] - Uploadé: gs://{BUCKET_NAME}/{gcs_path}")
    print(f"[OK] - Taille: {file_size_mb:.2f} MB")
    
    return f"gs://{BUCKET_NAME}/{gcs_path}"

def upload_folder_to_gcs(folder_path, gcs_folder="bronze", gcs_subfolder="", extensions=None):
    """Upload récursivement tous les fichiers d'un dossier vers GCS en préservant la structure
    
    Args:
        folder_path: Chemin du dossier local à uploader
        gcs_folder: Dossier de base dans GCS (défaut: "bronze")
        gcs_subfolder: Sous-dossier dans GCS (défaut: "")
        extensions: Liste des extensions à uploader (ex: [".csv", ".txt"]). Si None, upload tous les fichiers
    """
    folder_path = Path(folder_path)
    
    if not folder_path.exists():
        raise FileNotFoundError(f"Le dossier {folder_path} n'existe pas")
    
    # Normaliser les extensions (ajouter le point si absent, mettre en minuscule)
    if extensions:
        extensions = [ext.lower() if ext.startswith(".") else f".{ext.lower()}" for ext in extensions]
    
    bucket = storage_client.bucket(BUCKET_NAME)
    uploaded_files = 0
    total_size = 0
    skipped_files = 0
    
    print(f"[...] - Upload du dossier {folder_path.name} vers GCS...")
    if extensions:
        print(f"[...] - Extensions filtrées: {', '.join(extensions)}")
    
    # Parcourir récursivement tous les fichiers
    for file_path in folder_path.rglob("*"):
        if file_path.is_file():
            # Filtrer par extension si spécifié
            if extensions and file_path.suffix.lower() not in extensions:
                skipped_files += 1
                continue
            
            # Calculer le chemin relatif depuis le dossier source
            relative_path = file_path.relative_to(folder_path)
            
            # Construire le chemin GCS en préservant la structure
            if gcs_subfolder:
                gcs_path = f"{gcs_folder}/{gcs_subfolder}/{relative_path.as_posix()}"
            else:
                gcs_path = f"{gcs_folder}/{relative_path.as_posix()}"
            
            blob = bucket.blob(gcs_path)
            blob.upload_from_filename(str(file_path))
            
            file_size = file_path.stat().st_size
            total_size += file_size
            uploaded_files += 1
            
            print(f"  ✓ {relative_path.as_posix()}")
    
    total_size_mb = total_size / (1024 * 1024)
    print(f"[OK] - {uploaded_files} fichiers uploadés")
    if skipped_files > 0:
        print(f"[OK] - {skipped_files} fichiers ignorés (extension non autorisée)")
    print(f"[OK] - Taille totale: {total_size_mb:.2f} MB")
    print(f"[OK] - Dossier GCS: gs://{BUCKET_NAME}/{gcs_folder}/{gcs_subfolder if gcs_subfolder else folder_path.name}/")
    
    return uploaded_files



# 1 - Ingestion des données historiques de validation

In [35]:
# 1.1 - Récupération des liens des fichiers depuis l'API ile de de France Mobilités
URL = "https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/histo-validations-reseau-ferre/records"

response = requests.get(URL, timeout=60)
response.raise_for_status()

records = response.json().get("results", [])
print(f"Nombre de fichiers trouvés: {len(records)}")

df_metadata = pd.DataFrame([
    {
        'annee': int(r['annee']),
        'filename': r['reseau_ferre']['filename'],
        'url': r['reseau_ferre']['url']
    }
    for r in records
])

print("\nFichiers disponibles:")
display(df_metadata)

Nombre de fichiers trouvés: 10

Fichiers disponibles:


,annee,filename,url
0,2017,data-rf-2017.zip,https://data.iledefrance-mobilites.fr/api/expl...
1,2019,data-rf-2019.zip,https://data.iledefrance-mobilites.fr/api/expl...
2,2020,data-rf-2020.zip,https://data.iledefrance-mobilites.fr/api/expl...
3,2021,data-rf-2021.zip,https://data.iledefrance-mobilites.fr/api/expl...
4,2016,data-rf-2016.zip,https://data.iledefrance-mobilites.fr/api/expl...
5,2018,data-rf-2018.zip,https://data.iledefrance-mobilites.fr/api/expl...
6,2022,data-rf-2022.zip,https://data.iledefrance-mobilites.fr/api/expl...
7,2023,data-rf-2023.zip,https://data.iledefrance-mobilites.fr/api/expl...
8,2015,data-rf-2015.zip,https://data.iledefrance-mobilites.fr/api/expl...
9,2024,data-rf-2024.zip,https://data.iledefrance-mobilites.fr/api/expl...


In [36]:
# 1.2 - Téléchargement des fichiers
BASE_DIR = DATA_DIR / "histo-validations-reseau-ferre"
BASE_DIR.mkdir(parents=True, exist_ok=True)

for row in df_metadata.itertuples():
    year_dir = BASE_DIR / str(row.annee)
    year_dir.mkdir(exist_ok=True)
    zip_path = year_dir / row.filename

    print(f"[...] - Téléchargement {row.annee} ({row.filename})")
    try:
        response = requests.get(row.url, timeout=300)
        response.raise_for_status()
        
        zip_path.write_bytes(response.content)
        print(f"[OK] - Téléchargé")
        
        with ZipFile(zip_path) as zf:
            zf.extractall(year_dir)
        print(f"[OK] - Extrait dans {year_dir}")
        
        zip_path.unlink()
        
    except Exception as e:
        print(f"[Erreur] {row.annee}: {e}")

print(f"\n[OK] - Terminé: {BASE_DIR}")


[...] - Téléchargement 2017 (data-rf-2017.zip)
[OK] - Téléchargé
[OK] - Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2017
[...] - Téléchargement 2019 (data-rf-2019.zip)
[OK] - Téléchargé
[OK] - Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2019
[...] - Téléchargement 2020 (data-rf-2020.zip)
[OK] - Téléchargé
[OK] - Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2020
[...] - Téléchargement 2021 (data-rf-2021.zip)
[OK] - Téléchargé
[OK] - Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2021
[...] - Téléchargement 2016 (data-rf-2016.zip)
[OK] - Téléchargé
[OK] - Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2016
[...] - Téléchargement 2018 (data-rf-2018.zip)
[OK

In [37]:
# 1.3 - Upload vers GCS
print(f"\n[Upload] - Upload des données historiques vers GCS...")
upload_folder_to_gcs(BASE_DIR, gcs_folder="bronze", gcs_subfolder="histo-validations-reseau-ferre", extensions=[".csv", ".txt"])


[Upload] - Upload des données historiques vers GCS...
[...] - Upload du dossier histo-validations-reseau-ferre vers GCS...
[...] - Extensions filtrées: .csv, .txt
  ✓ 2022/data-rf-2022/2022_S2_PROFIL_FER.txt
  ✓ 2022/data-rf-2022/2022_S1_NB_FER.txt
  ✓ 2022/data-rf-2022/2022_S1_PROFIL_FER.txt
  ✓ 2022/data-rf-2022/2022_S2_NB_FER.txt
  ✓ 2024/2024_S1_PROFIL_FER.txt
  ✓ 2024/2024_S1_NB_FER.txt
  ✓ 2023/data-rf-2023/2023_S2_PROFIL_FER.txt
  ✓ 2023/data-rf-2023/2023_S1_NB_FER .txt
  ✓ 2023/data-rf-2023/2023_S2_NB_FER.txt
  ✓ 2023/data-rf-2023/2023_S1_PROFIL_FER.txt
  ✓ 2015/data-rf-2015/2015S2_NB_FER.csv
  ✓ 2015/data-rf-2015/2015S1_PROFIL_FER.csv
  ✓ 2015/data-rf-2015/2015S2_PROFIL_FER.csv
  ✓ 2015/data-rf-2015/2015S1_NB_FER.csv
  ✓ 2017/data-rf-2017/2017_S2_PROFIL_FER.txt
  ✓ 2017/data-rf-2017/2017_S2_NB_FER.txt
  ✓ 2017/data-rf-2017/2017S1_PROFIL_FER.txt
  ✓ 2017/data-rf-2017/2017S1_NB_FER.txt
  ✓ 2019/data-rf-2019/2019_S2_NB_FER.txt
  ✓ 2019/data-rf-2019/2019_S2_PROFIL_FER.txt
  ✓ 201

38

# 2 - Ingestion des données d'emplacement des gares

In [38]:
# 2.1 - Récupération du fichier CSV depuis l'API ile de de France Mobilités
dl_path_gares = download_parquet_from_idfm('emplacement-des-gares-idf')
upload_to_gcs(file_path=dl_path_gares, gcs_folder="bronze", gcs_subfolder="emplacement-des-gares-idf")

[...] - Téléchargement du fichier Parquet emplacement-des-gares-idf...
[OK] - Téléchargé: /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/emplacement-des-gares-idf/emplacement-des-gares-idf.parquet
[OK] - Taille: 0.20 MB
[...] - Upload de emplacement-des-gares-idf.parquet vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/emplacement-des-gares-idf/emplacement-des-gares-idf.parquet
[OK] - Taille: 0.20 MB


'gs://bronze-sncf-etl/bronze/emplacement-des-gares-idf/emplacement-des-gares-idf.parquet'

# 3 - Ingestion des données lignes

In [39]:
# 3.1 - Téléchargement direct du fichier Parquet depuis l'API IDFM
dl_path_lignes = download_parquet_from_idfm('referentiel-des-lignes')
upload_to_gcs(file_path=dl_path_lignes, gcs_folder="bronze", gcs_subfolder="referentiel-des-lignes")


[...] - Téléchargement du fichier Parquet referentiel-des-lignes...
[OK] - Téléchargé: /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/referentiel-des-lignes/referentiel-des-lignes.parquet
[OK] - Taille: 0.17 MB
[...] - Upload de referentiel-des-lignes.parquet vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/referentiel-des-lignes/referentiel-des-lignes.parquet
[OK] - Taille: 0.17 MB


'gs://bronze-sncf-etl/bronze/referentiel-des-lignes/referentiel-des-lignes.parquet'

# 4 - Ingestion des données Arrets

In [ ]:
# 4.1 - Téléchargement direct du fichier Parquet depuis l'API IDFM
dl_path_arrets = download_parquet_from_idfm('arrets')
upload_to_gcs(file_path=dl_path_arrets, gcs_folder="bronze", gcs_subfolder="arrets")

[...] - Téléchargement du fichier Parquet arrets...
[OK] - Téléchargé: /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/arrets/arrets.parquet
[OK] - Taille: 2.76 MB
[...] - Upload de arrets.parquet vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/arrets/arrets.parquet
[OK] - Taille: 2.76 MB


'gs://bronze-sncf-etl/bronze/arrets/arrets.parquet'

In [ ]:
# 5 - Ingestion Vacances Scolaires, Jours Fériés et Météo


In [51]:
# 5.1 - Vacances Scolaires
URL = "https://data.education.gouv.fr/explore/dataset/fr-en-calendrier-scolaire/download/?format=csv"
OUTPUT_FILE = "vacances_scolaires.csv"

vacances_dir = DATA_DIR / "vacances-scolaires"
vacances_dir.mkdir(exist_ok=True)
csv_path = vacances_dir / OUTPUT_FILE

print(f"[...] - Téléchargement des données de vacances scolaires...")
response = requests.get(URL, timeout=120)
response.raise_for_status()

csv_path.write_bytes(response.content)

file_size_mb = csv_path.stat().st_size / (1024 * 1024)
print(f"[OK] - Téléchargé: {csv_path}")
print(f"[OK] - Taille: {file_size_mb:.2f} MB")

# Upload vers GCS
upload_to_gcs(file_path=csv_path, gcs_folder="bronze", gcs_subfolder="vacances-scolaires")


[...] - Téléchargement des données de vacances scolaires...
[OK] - Téléchargé: /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/vacances-scolaires/vacances_scolaires.csv
[OK] - Taille: 0.23 MB
[...] - Upload de vacances_scolaires.csv vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/vacances-scolaires/vacances_scolaires.csv
[OK] - Taille: 0.23 MB


'gs://bronze-sncf-etl/bronze/vacances-scolaires/vacances_scolaires.csv'

In [53]:
# 5.2 - Jours Fériés (API Gouv) - Plusieurs années
start_year = 2015
end_year = datetime.now().year + 1

feries_dir = DATA_DIR / "jours-feries"
feries_dir.mkdir(exist_ok=True)

print(f"[...] - Téléchargement des jours fériés de {start_year} à {end_year}...")

for year in range(start_year, end_year + 1):
    URL = f"https://calendrier.api.gouv.fr/jours-feries/metropole/{year}.json"
    json_path = feries_dir / f"jours_feries_{year}.json"
    
    try:
        response = requests.get(URL, timeout=60)
        response.raise_for_status()
        
        feries = response.json()
        json_path.write_text(json.dumps(feries, ensure_ascii=False, indent=2), encoding="utf-8")
        
        print(f"[OK] - {year}: {len(feries)} jours fériés")
        
        # Upload vers GCS
        upload_to_gcs(file_path=json_path, gcs_folder="bronze", gcs_subfolder="jours-feries")
        
    except Exception as e:
        print(f"[Erreur] - {year}: {e}")

print(f"\n[OK] - Terminé: {feries_dir}")


[...] - Téléchargement des jours fériés de 2015 à 2026...
[OK] - 2015: 11 jours fériés
[...] - Upload de jours_feries_2015.json vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/jours-feries/jours_feries_2015.json
[OK] - Taille: 0.00 MB
[OK] - 2016: 11 jours fériés
[...] - Upload de jours_feries_2016.json vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/jours-feries/jours_feries_2016.json
[OK] - Taille: 0.00 MB
[OK] - 2017: 11 jours fériés
[...] - Upload de jours_feries_2017.json vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/jours-feries/jours_feries_2017.json
[OK] - Taille: 0.00 MB
[OK] - 2018: 11 jours fériés
[...] - Upload de jours_feries_2018.json vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/jours-feries/jours_feries_2018.json
[OK] - Taille: 0.00 MB
[OK] - 2019: 11 jours fériés
[...] - Upload de jours_feries_2019.json vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/jours-feries/jours_feries_2019.json
[OK] - Taille: 0.00 MB
[OK] - 2020: 11 j